In [ ]:
# 基于iwencai flag的超短线策略
# 轮训iwencai_flag_history以及ths_kday的数据并统计，每个flag出现的情况下：
# 1. 第二天的涨幅
# 2. 后五天的涨幅
# 3. 排除涨停的情况下第二天的涨幅
# 将结果保存在iwencai_flag_result表中，格式为 {"flag":"...", "oneday":[...], "onedayf":[...], "fiveday":[...]}
from opentrader.core.ticker import *
from opentrader.common.db import db_ot
from opentrader.common.utils import d2dt
from opentrader.agents.ths.api import symbol_convert3
from pymongo.errors import *
from datetime import datetime, timedelta

start_date = gen_time('2015-01-01 00:00:00').date()
end_date = gen_time('2015-12-17 00:00:00').date()
date = start_date
dates = []
while date <= end_date:
    if TradeCalendar.check_date(date):
        dates.append(date)
    date += timedelta(1)

In [ ]:
# 将k线数据缓存在内存中
kday_cache = {} # {'...symbol...':[{}, {}, {}]}
cur_year = 0
for date in dates:
    if date.year != cur_year:
        print(date.year)
        cur_year = date.year
    else:
        print('.', end='')
    result = db_ot.ths_kday.find({'date':d2dt(date)})
    for each in result:
        if each['symbol'] not in kday_cache:
            kday_cache[each['symbol']] = []
        kday_cache[each['symbol']].append(each)

In [ ]:
# 从k线缓存获取数据的工具函数
# only the first day has a calculated "percent" value
def get_k_day_from_date(symbol, date, count=10):
    if symbol not in kday_cache:
        return None
    for (i, each) in enumerate(kday_cache[symbol]):
        if each['date'] == d2dt(date):
            rtn = kday_cache[symbol][i:(i+count)]
            if i > 0:
                rtn[0]['percent'] = (float(rtn[0]['close']) / float(kday_cache[symbol][i-1]['close']) - 1) * 100
            else:
                rtn[0]['percent'] = 0.0
            return rtn
    return None
    #result = db_ot.ths_kday.find({'symbol':symbol, 'date':{'$gte':d2dt(date)}}).sort('date')
    #return list(result)

#print(get_k_day_from_date("SH601139", datetime(2015, 1, 5, 0, 0)))

In [ ]:
# 对每个flag统计每个样本的表现
flag_stats = {} # {"...flag...":{"oneday":[], "fiveday":[], "onedayf":[], "fivedayf":[], ...}

# 根据日期查询并统计
import json
cur_year = 0
for date in dates:
    if date.year != cur_year:
        print(date.year)
        cur_year = date.year
    else:
        print('.', end='')
    f = open('iwencai_flags.json', 'r')
    try:
        flags = json.loads(f.read())
    except Exception as e:
        print(e)
        flags = []
    i = 0
    for flag in flags:
        result = db_ot.iwencai_flag_history.find_one({'flag':flag, 'date':d2dt(date)})
        if not result:
            continue
        if flag not in flag_stats:
            flag_stats[flag] = {"oneday":[], "fiveday":[], "onedayf":[], "fivedayf":[]}
        for each_symbol in result['symbols']:
            sym = symbol_convert3(each_symbol)
            kday = get_k_day_from_date(sym, date)
            if not kday or len(kday) < 2:
                continue
            if kday[0]['date'] != d2dt(date):
                continue
            oneday = (float(kday[1]['close']) / float(kday[0]['close']) - 1) * 100
            flag_stats[flag]['oneday'].append(oneday)
            if len(kday) >= 6:
                fiveday = (float(kday[5]['close']) / float(kday[0]['close']) - 1) * 100
                flag_stats[flag]['fiveday'].append(fiveday)
                if kday[0]['percent'] <= 9.8:
                    flag_stats[flag]['fivedayf'].append(fiveday)
            if kday[0]['percent'] <= 9.8:
                flag_stats[flag]['onedayf'].append(oneday)
        i += 1
f = open('iwencai_flag_stats.json', 'w')
f.write(json.dumps(flag_stats))
f.close()

In [ ]:
print(len(flag_stats))

In [ ]:
# 计算和呈现不同flag的五日表现
result = []
for flag,each in flag_stats.items():
    avr = 0.0
    if not each['fivedayf']:
        continue
    for percent in each['fivedayf']:
        avr += percent
    avr = avr / len(each['fivedayf'])
    result.append({'flag':flag, 'avr':avr, 'count':len(each['fivedayf'])})

sorted_x = sorted(result, key=lambda x:x['avr'])
for each in sorted_x:
    print("flag: %s, count: %d, mean: %f" % (each['flag'], each['count'], each['avr']))

In [ ]:
# 计算和呈现不同flag的单日表现
result = []
for flag,each in flag_stats.items():
    avr = 0.0
    if not each['onedayf']:
        continue
    for percent in each['onedayf']:
        avr += percent
    avr = avr / len(each['onedayf'])
    result.append({'flag':flag, 'avr':avr, 'count':len(each['onedayf'])})

sorted_x = sorted(result, key=lambda x:x['avr'])
for each in sorted_x:
    print("flag: %s, count: %d, mean: %f" % (each['flag'], each['count'], each['avr']))

In [ ]:
# 计算历史大盘指数并与上证指数比较
oneday_result = {}
for flag,each in flag_stats.items():
    avr = 0.0
    if not each['oneday']:
        continue
    for percent in each['oneday']:
        avr += percent
    avr = avr / len(each['oneday'])
    oneday_result[flag] = {'avr':avr, 'count':len(each['oneday'])}
    
import json
f = open('iwencai_flags_withtype.json', 'r')
flags_withtype = json.loads(f.read())
f.close()
indexs = []   # 指数1: 根据buy与sale类型flag次数累计后求平均 
indexs2 = []  # 指数2: 根据所有flag每次出现求和平均后的大盘期望
szzs = []
last_sh = 3000.0
for date in dates: 
    index = 0
    index2 = 0
    totalflags = 0
    for key in ("buy", "sale", "event", "zxst"):
        for flag in flags_withtype[key]:
            result = db_ot.iwencai_flag_history.find_one({'flag':flag, 'date':d2dt(date)})
            if not result:
                continue
            if key == 'buy':
                index += len(result['symbols'])
            elif key == 'sale':
                index -= len(result['symbols'])
            if flag in oneday_result:
                index2 += oneday_result[flag]['avr'] * len(result['symbols'])
            totalflags += len(result['symbols'])
            
    index = index/2666
    index2 = index2/totalflags
    indexs.append(index)
    indexs2.append(index2)
    sh = float(db_ot.ths_kday.find_one({'symbol':'SH000001', 'date':d2dt(date)})['close'])
    szzs.append(sh)
    print("%s, index:%f, index2:%f, szzs:%f" % (date, index, index2, (sh/last_sh-1.0)*100))
    last_sh = sh
    # 重要股灾日： 5.5, 5.28, 6.16, 6.26, 7.27, 8.18, 9.15, 10.21, 11.27

In [ ]:
# 计算每天期望值前五名的个股，并记录第二天的平均涨幅
import json, operator
f = open('iwencai_flags.json', 'r')
flags = json.loads(f.read())
f.close()
onedayf_result = {}

for flag,each in flag_stats.items():
    avr = 0.0
    if not each['onedayf']:
        continue
    for percent in each['onedayf']:
        avr += percent
    avr = avr / len(each['onedayf'])
    onedayf_result[flag] = {'avr':avr, 'count':len(each['onedayf'])}

index2_max = max(indexs2)
date_performance = []
date_performance2 = [] # 考虑仓位变化的持仓股票performance
for (idate, date) in enumerate(dates):
    position_counts = round(indexs2[idate]/index2_max/0.2)
    symbol_means = {}
    symbol_max_flag = {}
    for flag in flags:
        if flag in ('停牌','涨停'):# 此处过滤 “涨停” 是为了滤掉ST股票的影响
            continue
        if '光头' in flag:
            continue
        result = db_ot.iwencai_flag_history.find_one({'flag':flag, 'date':d2dt(date)})
        if not result:
            continue
        for symbol in result['symbols']:
            if symbol not in symbol_means:
                symbol_means[symbol] = []
            try:
                symbol_means[symbol].append(onedayf_result[flag]['avr'])
                if symbol not in symbol_max_flag or onedayf_result[flag]['avr'] > onedayf_result[symbol_max_flag[symbol]]['avr']:
                    symbol_max_flag[symbol] = flag
            except KeyError:
                continue
    for symbol,means in symbol_means.items():
        symbol_means[symbol] = sum(means)/len(means)
    symbol_means_sorted = sorted(symbol_means.items(), key=operator.itemgetter(1), reverse=True)
    performance = []
    stocks = []
    i = 0
    for (symbol, mean) in symbol_means_sorted:
        sym = symbol_convert3(symbol)
        #if sym.startswith("SZ300"):
        #    continue
        kdays = get_k_day_from_date(sym, date)
        if kdays is None or len(kdays) < 2 or float(kdays[0]['percent']) > 9.8:
            continue
        if idate > 0 and float(kdays[0]['percent']) == 0.0: #第一天发行的股票
            continue
        if kdays[0]['date'] != d2dt(date): # 停牌
            continue
        else:
            performance.append((float(kdays[1]['close']) / float(kdays[0]['close']) - 1)*100)
            stocks.append((sym, symbol_max_flag[symbol]))
        i += 1
        if i >= 5:
            break
    # 重要股灾日： 5.5, 5.28, 6.16, 6.26, 7.27, 8.18, 9.15, 10.21, 11.27
    if len(performance) == 0:
        date_performance.append(0.0)
        date_performance2.append(0.0)
        print("%s: zero result" % (date))
    else:
        date_performance.append(sum(performance) / len(performance))
        position_counts = min(position_counts, len(performance))
        if position_counts == 0:
            date_performance2.append(0.0)
        else:
            date_performance2.append(sum(performance[:position_counts]) / position_counts)
        print("%s: available stocks: %s, mean: %f, stocks: %s, position: %f" % (date, len(performance), sum(performance) / len(performance), stocks, round(indexs2[idate]/index2_max/0.2)*0.2))


In [ ]:
f = open('iwencai_flag_performance_onedayf.json', 'w')
f.write(json.dumps(onedayf_result))
f.close()

In [ ]:
fund_basic = []
init = 1.0
for each in date_performance:
    init = init * (1.0 + each/100.0)
    fund_basic.append(init)

fund = []
positions = []
init = 1.0
trade_days = 0
for (i, each) in enumerate(date_performance2):
    if indexs2[i] < 0.0:
        fund.append(init)
        positions.append(0.0)
        continue
    if each < -4.0:
        each = -4.0
    #init = init * (1.0 + each/100.0)
    init *= 0.998
    position = float(round(indexs2[i]/index2_max/0.2)*0.2)
    init = init * (1.0 - position) + init * position * (1.0 + each/100.0)
    fund.append(init)
    positions.append(position)
    trade_days += 1
    
print(index2_max)
print(init)
print(len(fund))
print(len(dates))
print(len(indexs))
print(trade_days)

%matplotlib inline
from opentrader.core.crawler import *
from datetime import datetime
from opentrader.jupyter.lib.plot import *
from opentrader.ceres.account import Account
from opentrader.ceres.trade import *

draw_line("index", dates, [{"value":indexs}])
draw_line("index2", dates, [{"value":indexs2}])
draw_line("position", dates, [{"value":positions}])
draw_line("performance", dates, [{"value":fund_basic}])
draw_line("performance", dates, [{"value":fund}])